# Automated Hyperparameter Optimization Training using WMLA API

&nbsp;
&nbsp;
&nbsp;
&nbsp;
&nbsp;


![options](https://github.com/IBM/wmla-learning-path/raw/master/shared-images/WMLA-RestAPI-Demo.png)



![SpectrumComputeFamily_Conductor-HorizontalColorWhite.png](https://raw.githubusercontent.com/IBM/wmla-learning-path/master/shared-images/hpo.png)

In this notebook you will learn how to submit a model and dataset to the Watson Machine Learning Accelerator (WMLA) API to run Hyper Parameter Optimization (HPO).  

For this notebook you will use a model and dataset that have already been setup to leverage the API.  For details on the API see this link in the Knowledge Center (KC).

[API Documentation](https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/deeplearning.html)

In [11]:
!wget https://github.com/IBM/wmla-learning-path/raw/master/datasets/pytorch-mnist-hpo.modelDir.tar

--2020-07-16 20:15:46--  https://github.com/IBM/wmla-learning-path/raw/master/datasets/pytorch-mnist-hpo.modelDir.tar
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/wmla-learning-path/master/datasets/pytorch-mnist-hpo.modelDir.tar [following]
--2020-07-16 20:15:47--  https://raw.githubusercontent.com/IBM/wmla-learning-path/master/datasets/pytorch-mnist-hpo.modelDir.tar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2587 (2.5K) [application/octet-stream]
Saving to: 'pytorch-mnist-hpo.modelDir.tar.1'

100%[======================================>] 2,587       --.-K/s   in 0s      

2020-07-16 20:15:47 (29.4 MB/s) - 'pytorch-mnist-hp

## Imports

To use the WMLA API, we will be using the python requests library

In [12]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import json
import time
import urllib
import pandas as pd
import os,sys
import tarfile
import tempfile
from IPython.display import clear_output
import time
import pprint

# utility print function
def nprint(mystring) :
    print("**{}** : {}".format(sys._getframe(1).f_code.co_name,mystring))

# utility makedir
def makeDirIfNotExist(directory) :
    if not os.path.exists(directory):  
        nprint("Making directory {}".format(directory))
        os.makedirs(directory) 
    else :
        nprint("Directory {} already exists .. ".format(directory))


## Environment details and Project Config:

In [13]:
def getconfig(cfg_in={}):
    cfg = {}
    cfg["master_host"] = 'https://dse-ac922h.cpolab.ibm.com'
    cfg["dli_rest_port"] = '9243'
    cfg["sc_rest_port"] = '8643'
    cfg["num_images"] = {"train":200,"valid":20,"test":20}
    # ==== CLASS ENTER User login details below =====
    cfg["wmla_user"] = 'Admin'  # <=enter your id here
    cfg["wmla_pwd"] = 'Admin'  # <=enter your pwd here
    # ==== CLASS ENTER User login details above =====
    cfg["sig_name"]  = 'SIG-DSE'
    cfg["code_dir"] = "/home/wsuser/works/pytorch_hpo"

    # overwrite configs if passed
    for (k,v) in cfg_in.items() :
        nprint("Overriding Config {}:{} with {}".format(k,cfg[k],v))
        cfg[k] = v
    return cfg

# cfg is used as a global variable throughout this notebook
cfg=getconfig()

In [14]:
# REST call variables
commonHeaders = {'Accept': 'application/json'}

# Use closures for cfg for now ..
def get_tmp_dir() :
    #return "/gpfs/home/s4s004/"+cfg["wmla_user"]+"/2020-05-wmla/tmp"
    return "/home/wsuser/work"
def get_tar_file() :
    #return get_tmp_dir() + "/" + cfg["wmla_user"]+".modelDir.tar"
    return get_tmp_dir() + "/pytorch-mnist-hpo.modelDir.tar"

#get api endpoint
def get_ep(mode="sc") :
    if mode=="sc" :
        sc_rest_url =  cfg["master_host"] +':'+ cfg["sc_rest_port"] +'/platform/rest/conductor/v1'
        return sc_rest_url
    elif(mode=="dl") :
        dl_rest_url = cfg["master_host"] +':'+cfg["dli_rest_port"] +'/platform/rest/deeplearning/v1'
        return dl_rest_url
    else :
        nprint("Error mode : {} not supported".format(mode))

def myauth():
    return(cfg["wmla_user"],cfg["wmla_pwd"])

print ("SC API Endpoints : {}".format(get_ep("sc")))
print ("DL API Endpoints : {}".format(get_ep("dl")))
print (myauth())
print (get_tar_file())
#myauth = (wmla_user, wmla_pwd)

# Setup Requests session
req = requests.Session()

SC API Endpoints : https://dse-ac922h.cpolab.ibm.com:8643/platform/rest/conductor/v1
DL API Endpoints : https://dse-ac922h.cpolab.ibm.com:9243/platform/rest/deeplearning/v1
('Admin', 'Admin')
/home/wsuser/work/pytorch-mnist-hpo.modelDir.tar


## Health Check

Check if there is any existing hpo tasks and also verify the platform health

Rest API: **GET platform/rest/deeplearning/v1/hypersearch**
- Description: Get all the hpo task that the login user can access.
- OUTPUT: A list of hpo tasks and each one with the same format which can be found in the api doc.

In [15]:
def hpo_health_check():
    getTuneStatusUrl = get_ep("dl") + '/hypersearch'
    nprint ('getTuneStatusUrl: %s' %getTuneStatusUrl)
    r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=myauth())
    
    if not r.ok:
        nprint('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
    else:
        if len(r.json()) == 0:
            nprint('There is no hpo task been created')
        for item in r.json():
            nprint('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
            #print('Best:%s'%json.dumps(item.get('best'), sort_keys=True, indent=4))

hpo_health_check()


**hpo_health_check** : getTuneStatusUrl: https://dse-ac922h.cpolab.ibm.com:9243/platform/rest/deeplearning/v1/hypersearch
**hpo_health_check** : Hpo task: Admin-hpo-16650377032674085, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16652311451466531, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16709092768652198, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16783740796460860, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16797156948888025, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16899927798480429, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-16901184356247510, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-17306812964985375, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-17310278053357251, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-17550982147791472, State: FINISHED
**hpo_health_check** : Hpo task: Admin-hpo-17555313438664566, State: FINISHED
**hpo_health_check**

## Changes to your code

**Developer** Note The WMLA framework requires 2 changes to your code to support the HPO API, these are :

* Inject hyper-parameters for the sub-training during search
* Retrieve sub-training result metric

Note that the code sections below show a comparison between the "before" and "HPO enabled" versions of the code using `diff`.


1. Import the dependent libararies:

&nbsp;
&nbsp;
![image1](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_0.png)
&nbsp;
&nbsp;

2. Get the WMLA cluster DLI_DATA_FS, RESULT_DIR and LOG_DIR for the hpo training job. The DLI_DATA_FS could be used for shared data placement, the RESULT_DIR can be used for final model saving, and the LOG_DIR can be used for user logs and monitoring.

&nbsp;
Note: DLI_DATA_FS is set when install DLI cluster; RESULT_DIR and LOG_DIR is generated by WMLA for each HPO experiment.

&nbsp;
&nbsp;
![image1](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_1.png)
&nbsp;
&nbsp;

3. Replace the hyperparameter definition code by reading hyperparameters from the `config.json` file. the `config.json` is generated by WMLA HPO, which contains a set of hyperparameter candidates for each tuning jobs. The hyperparameters and the search space is defined when submitting the HPO task. For example, here the hyperparameter learning_rate is set to tune:

&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_2.png)

&nbsp;
Then you could use the hyperparameter you get from `config.json` where you want:
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_2_2.png)
&nbsp;
&nbsp;

4.  Write the tuning result into `val_dict_list.json` under RESULT_DIR. WMLA HPO will read this file for each tuning job to get the metric values. Define a test_metrics list to store all metric values and pass the epoch parameter to the test function. Then you could add the metric values to the test_metrics list during the training test process. Please note that the metric names should be specified when submitting the HPO task, and be consitent with the code here.
&nbsp;
For example, the HPO task submit request, `loss` will be used as the objective metric the tuning will try to minimize the `loss`:

```
'algoDef': # Define the parameters for search algorithms  
{
    # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband  
    'algorithm': 'Random',   
    # Name of the target metric that we are trying to optimize when searching hyper-parameters.
    # It is the same metric name that the model update part 2 trying to dump.
    'objectiveMetric' : 'loss',
    # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
    # make the above objectiveMetric as small as possible, maximize means the opposite.
    'objective' : 'minimize',
    ...
}
```
&nbsp;
The code change:

&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_3.png)
&nbsp;
&nbsp;

5. After the training complete, write the metric list into the `val_dict_list.json` file. 
&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_5.png)
&nbsp;
&nbsp;



### Launch HPO task

Here we package up our model to send to the API for HPO.  Lets see how this works ...



REST API: **POST /platform/rest/deeplearning/v1/hypersearch**
- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model files tar package, ending with `.modelDir.tar`
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


#### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

REST API expects a modelDir.tar with the model code inside ..


In [16]:
files = {'file': open(get_tar_file(), 'rb')}
print("Files : {}".format(files))

Files : {'file': <_io.BufferedReader name='/home/wsuser/work/pytorch-mnist-hpo.modelDir.tar'>}


#### Construct POST request data

**hpo_input** will be a Python dict or json format as below, convert to string when calling REST.

In [17]:
# Note, this 
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': cfg["sig_name"],

            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            #
            'args': '--exec-start PyTorch --cs-datastore-meta type=fs --python-version 3.6\
                     --gpuPerWorker 1 --model-main pytorch_mnist_HPO.py --model-dir pytorch_hpo\
                     --debug-level debug'
                
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband, ExperimentGridSearch
            'algorithm': 'Random', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': 60,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 4,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 4, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

#### Submit the Post request

Submit hpo task through the Post call and a hpo name/id as string format will get back.

**Note**:This cannot be submitted twice.. you need to rebuild the tar file prior to resubmitting

In [18]:
def submit_job():
    startTuneUrl=get_ep('dl') + '/hypersearch'
    nprint("startTuneUrl : {}".format(startTuneUrl))
    nprint("files : {}".format(files))
    nprint("myauth() : {}".format(myauth()))
    #print("hpo_job_id : {}".format(hpo_job_id))
    r = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=myauth())
    hpo_name=None
    if r.ok:
        hpo_name = r.json()
        print ('\nModel submitted successfully: {}'.format(hpo_name))
        
    else:
        print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))
    return hpo_name

hpo_job_id = submit_job()
print("hpo_job_id : {}".format(hpo_job_id))

**submit_job** : startTuneUrl : https://dse-ac922h.cpolab.ibm.com:9243/platform/rest/deeplearning/v1/hypersearch
**submit_job** : files : {'file': <_io.BufferedReader name='/home/wsuser/work/pytorch-mnist-hpo.modelDir.tar'>}
**submit_job** : myauth() : ('Admin', 'Admin')

Model submitted successfully: Admin-hpo-20596057193264056
hpo_job_id : Admin-hpo-20596057193264056


In [19]:
getHpoUrl = get_ep('dl') +'/hypersearch/'+ hpo_job_id
pp = pprint.PrettyPrinter(indent=2)

keep_running=True
rr=10
res=None
while(keep_running):
    res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=myauth())
    experiments=res.json()['experiments']
    experiments = pd.DataFrame.from_dict(experiments)
    pd.set_option('max_colwidth', 120)
    clear_output()
    print("Refreshing every {} seconds".format(rr))
    display(experiments)
    pp.pprint(res.json())
    if(res.json()['state'] not in ['SUBMITTED','RUNNING']) :
        keep_running=False
    time.sleep(rr)

Refreshing every 10 seconds


,appId,driverId,endTime,hyperParams,id,maxiteration,metricVal,startTime,state
0,Admin-20596059577795518-282955637,driver-20200716161930-0097-7194fc7f-9631-4492-b8ab-0545f937b740,2020-07-16 16:23:57,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.03346157010939412'}]",0,0,0.068464,2020-07-16 16:19:30,FINISHED
1,Admin-20596065414636534-535084807,driver-20200716161936-0098-a67f7c4d-218c-44ac-8752-ac0ba21204e1,2020-07-16 16:23:57,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.0326022758934446'}]",1,0,0.069694,2020-07-16 16:19:36,FINISHED
2,Admin-20596071214210818-1914402851,driver-20200716161942-0099-b0811c14-5bdc-4dc2-800f-1d5769ab1399,2020-07-16 16:23:57,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.009658941961207125'}]",2,0,0.156826,2020-07-16 16:19:42,FINISHED
3,Admin-20596077084077847-1161107473,driver-20200716161948-0100-95ed657f-500f-41b3-99b7-f78f9b206af3,2020-07-16 16:23:58,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.06235879973901014'}]",3,0,0.049116,2020-07-16 16:19:48,FINISHED


{ 'best': { 'appId': 'Admin-20596077084077847-1161107473',
            'driverId': 'driver-20200716161948-0100-95ed657f-500f-41b3-99b7-f78f9b206af3',
            'endTime': '2020-07-16 16:23:58',
            'hyperParams': [ { 'dataType': 'double',
                               'fixedVal': '0.06235879973901014',
                               'name': 'learning_rate',
                               'userDefined': False}],
            'id': 3,
            'maxiteration': 0,
            'metricVal': 0.049116130065917966,
            'startTime': '2020-07-16 16:19:48',
            'state': 'FINISHED'},
  'complete': 4,
  'createtime': '2020-07-16 16:19:27',
  'creator': 'Admin',
  'duration': '00:04:31',
  'experiments': [ { 'appId': 'Admin-20596059577795518-282955637',
                     'driverId': 'driver-20200716161930-0097-7194fc7f-9631-4492-b8ab-0545f937b740',
                     'endTime': '2020-07-16 16:23:57',
                     'hyperParams': [ { 'dataType': 'double',
     

### Show the Best Result of HPO Job

In [10]:
# Lets query our result to see what happened during HPO training!

#res.ok
#res.json()
#print(type(res))
#print(dir(res))
#print(json.dumps(res.json(), indent=4, sort_keys=True))
        
print('Hpo task %s completes with state %s'%(hpo_job_id, res.json()['state']))
print("Best HPO result ...")
res.json()["best"]


Hpo task Admin-hpo-18774434951753245 completes with state FINISHED
Best HPO result ...


{'id': 3,
 'hyperParams': [{'name': 'learning_rate',
   'dataType': 'double',
   'userDefined': False,
   'fixedVal': '0.08731352060122094'}],
 'state': 'FINISHED',
 'metricVal': 0.041620475196838376,
 'maxiteration': 0,
 'appId': 'Admin-18774455706485225-1434488386',
 'driverId': 'driver-20200625141926-0088-b26b4e5c-9ead-481f-b5a8-1b36b1689c13',
 'startTime': '2020-06-25 14:19:26',
 'endTime': '2020-06-25 14:23:36'}

#### Notebook Complete 
Congratulations, you have completed our demonstration of using WMLA for distributed hyperparameter optimization search